In [19]:
import numpy as np
import pandas as pd

In [20]:
# zaimportowanie danych
df = pd.read_csv("winequality_white.csv", sep = ";" , encoding= 'utf-8')

In [21]:
# wyświetlenie pierwszy 5 rekordów
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.05,30,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186.0,0.9956,3.19,0.4,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186.0,0.9956,3.19,0.4,9.9,6


In [22]:
# 4898 rekordów, 12 zmiennych z czego jedna wyjściowa 
df.shape

(4898, 12)

In [23]:
# Inforamcje o zbiorze, nazwa kolumny, ile niepustych wartości, typ danych
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   object 
 5   free sulfur dioxide   4898 non-null   object 
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   object 
 10  alcohol               4895 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(8), int64(1), object(3)
memory usage: 459.3+ KB


In [24]:
# Obróbka danych

# Usunięcie duplikatów z DF, True, ponieważ usuwamy rekordy z aktualnego DF
df.drop_duplicates(inplace=True)

# Dla każdej kolumny danych typu 'object' usuwamy spacje, znak '_' z końca stringów, 
# sprawdzamy czy wszystkie znaki w danej kolumnie są wartościami numerycznymi, jeśli tak zamieniamy typ danych na float
for column in df.columns:
  if df[column].dtype == 'object':
    df[column] = df[column].str.replace(' ', '')
    df[column] = df[column].str.rstrip('_')
    if df[column].str.replace('.', '').str.isnumeric().all():
            df[column] = df[column].astype(float)

In [25]:
# Sprawdzamy wynik zmian
print(df.info())
print(df.head())
# wniosek: kolumna alkohol zawiera 3 nulle

<class 'pandas.core.frame.DataFrame'>
Index: 3963 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         3963 non-null   float64
 1   volatile acidity      3963 non-null   float64
 2   citric acid           3963 non-null   float64
 3   residual sugar        3963 non-null   float64
 4   chlorides             3963 non-null   float64
 5   free sulfur dioxide   3963 non-null   float64
 6   total sulfur dioxide  3963 non-null   float64
 7   density               3963 non-null   float64
 8   pH                    3963 non-null   float64
 9   sulphates             3963 non-null   float64
 10  alcohol               3960 non-null   float64
 11  quality               3963 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 402.5 KB
None
   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.0              0.27         0.36 

In [26]:
# Sprawdzamy unikalne wartości kolumny alcohol
print(df['alcohol'].unique())
# Znajdujemy wartości nan, ujemne, odstające (większe od 100),

[   8.8           9.5          10.1           9.9           9.6
   11.           12.            9.7          10.8          12.4
   11.4          -9.6          12.8          11.3          10.5
    9.3          10.           10.4          11.6          12.3
   10.2           9.           11.2           8.6           9.4
    9.8          11.7          10.9           9.1           8.9
   10.3          12.6          10.7          12.7          10.6
    9.2           8.7          11.5                  nan   11.8
   12.1          11.1           8.5          12.5          11.9
   12.2         -10.           12.9        1000.           13.9
   14.           13.5          13.3          13.2          13.7
   13.4          13.            8.           13.1          13.6
    8.4          14.2          11.94         12.89333333   11.46666667
   10.98         10.03333333   11.43333333   10.53333333    9.53333333
   10.93333333   11.36666667   11.33333333   11.06666667    9.73333333
   11.05          9

In [27]:
# Zamienamy wartości spoza zakresu od 0 do 100 na wartości nan, a następnie usuwamy zawierające je rekordy z df
df.loc[(df['alcohol'] < 0) | (df['alcohol'] > 100), 'alcohol'] = np.nan
df = df.dropna(subset=['alcohol'])

In [ ]:
# Sprawdzamy czy korekta wartości w kolumnie alcohol przebiegła pomyślnie
print(df['alcohol'].unique())

REGRESJA LASSO - JEDNAK NIE

In [29]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [30]:
# Dzielimy dane na zbiór testowy i zbiór uczący
X = df.drop(columns=["quality"])
y = df["quality"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Dokonujemy standardyzacji danych wejściowych, ma to na celu lepsze działanie modelu, jeśli dane są oparte o tę samą skalę
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [32]:
# Inicjacja modelu
lasso = Lasso(alpha=0.1)  # Ustawienie wartości alpha, która kontroluje siłę regularyzacji
# Trenowanie modelu
lasso.fit(X_train_scaled, y_train)

Lasso(alpha=0.1)

In [33]:
# Ocena modelu, wykonana za pomocą błędu średniokwadratowego (RMSE). Im wyższa wartość, tym model lepiej dopasowny do danych
train_rmse = mean_squared_error(y_train, lasso.predict(X_train_scaled), squared=False)
test_rmse = mean_squared_error(y_test, lasso.predict(X_test_scaled), squared=False)

print(f"RMSE dla zbioru uczącego: {train_rmse}")
print(f"RMSE dla zbioru testowego: {test_rmse}")


RMSE dla zbioru uczącego: 0.7895367145683293
RMSE dla zbioru testowego: 0.7296439408580088


c:\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


DRZEWA DECYZYJNE

In [1]:
from sklearn import datasets
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt

data = pd.read_csv("winequality_white.csv", sep = ";" , encoding= 'utf-8')

In [2]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.05,30,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47,186.0,0.9956,3.19,0.4,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47,186.0,0.9956,3.19,0.4,9.9,6
